<a href="https://colab.research.google.com/github/nosadchiy/public/blob/main/EOQ_CR_Dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal (prompt)
For EOQ base: Create an interactive Economic Order Quantity (EOQ) dashboard that allows users to input annual demand, holding cost per unit per year, and ordering cost per order, then calculates and displays the EOQ, number of orders per year, total annual holding cost, total annual ordering cost, and total annual inventory cost, and visualizes the inventory trajectory over time with an interactive plot including hover functionality.


To add uncertainty: I added two more input widgets: lead time and standard deviation of annual demand. Use values from those widgets to incorporate demand uncertainty into the model. Assume normal distribution for the demand with the mean equal to the annual demand. Modify the model to include safety stock and reorder point computation. Assume the continuous inventory review policy. Simulate inventory trajectory under uncertainty and update the inventory trajectory graph. Keep track of stockouts.


In [57]:
# @title [Imports]
import ipywidgets as widgets
from IPython.display import display

In [58]:
# @title [Define Input Widgets]
demand_widget = widgets.FloatText(
    value=1000.0,
    description='Annual Demand (D):',
    disabled=False,
    min=0
)
holding_cost_widget = widgets.FloatText(
    value=2.0,
    description='Holding Cost (H):',
    disabled=False,
    min=0
)
ordering_cost_widget = widgets.FloatText(
    value=100.0,
    description='Ordering Cost (S):',
    disabled=False,
    min=0
)

leadtime_widget = widgets.FloatText(
    value=10.0,
    description='Lead Time (L):',
    disabled=False,
    min=0
)

sd_widget = widgets.FloatText(
    value=100.0,
    description='SD Annual Demand (Sigma):',
    disabled=False,
    min=0
)


# Display the widgets using VBox for a vertical layout
eoq_input_form = widgets.VBox([
    demand_widget,
    holding_cost_widget,
    ordering_cost_widget, sd_widget, leadtime_widget
])
display(eoq_input_form)



In [59]:
# @title [Calc EOQ]
import math

def calculate_eoq_metrics(demand, holding_cost, ordering_cost):
    if demand <= 0 or holding_cost <= 0 or ordering_cost <= 0:
        print("All input values must be positive.")
        return

    # Calculate EOQ
    eoq = math.sqrt((2 * demand * ordering_cost) / holding_cost)

    # Calculate Number of orders per year
    num_orders_per_year = demand / eoq

    # Calculate Total annual holding cost
    total_annual_holding_cost = (eoq / 2) * holding_cost

    # Calculate Total annual ordering cost
    total_annual_ordering_cost = (demand / eoq) * ordering_cost

    # Calculate Total annual inventory cost
    total_annual_inventory_cost = total_annual_holding_cost + total_annual_ordering_cost

    # Display the calculated metrics
    print(f"Economic Order Quantity (EOQ): {eoq:.2f}")
    print(f"Number of Orders per Year: {num_orders_per_year:.2f}")
    print(f"Total Annual Holding Cost: {total_annual_holding_cost:.2f}")
    print(f"Total Annual Ordering Cost: {total_annual_ordering_cost:.2f}")
    print(f"Total Annual Inventory Cost: {total_annual_inventory_cost:.2f}")



In [60]:
# @title [Generate  Trajectory]

import math
import numpy as np # Import numpy for normal distribution sampling

def generate_inventory_trajectory(eoq, daily_demand, sd_daily_demand, lead_time, reorder_point, simulation_period=365):
    """
    Generates inventory levels over a specified period with demand uncertainty and order placement logic.

    Args:
        eoq (float): Economic Order Quantity.
        daily_demand (float): Average daily demand.
        sd_daily_demand (float): Standard deviation of daily demand.
        lead_time (int): Lead time in days for new orders.
        reorder_point (float): Inventory level at which a new order is placed.
        simulation_period (int): The total period (e.g., in days) for the simulation.

    Returns:
        tuple: A tuple containing three lists: time_points, inventory_levels, and stockout_flags.
    """
    if eoq <= 0 or daily_demand < 0 or sd_daily_demand < 0 or lead_time < 0 or simulation_period <= 0:
        print("Input values must be non-negative, and eoq/simulation_period must be positive.")
        return [], [], []

    current_inventory = eoq # Start with an initial order just arrived
    orders_in_transit = [] # List of tuples (arrival_day, order_quantity)

    time_points = []
    inventory_levels = []
    stockout_flags = [] # True if stockout occurred on that day, False otherwise

    for day in range(simulation_period):
        # 1. Sample daily demand
        # Ensure sampled demand is not negative
        sampled_demand = max(0, np.random.normal(daily_demand, sd_daily_demand))

        # 2. Process any incoming orders
        # Create a new list to avoid modifying orders_in_transit during iteration
        new_orders_in_transit = []
        for arrival_day, order_quantity in orders_in_transit:
            if arrival_day == day:
                current_inventory += order_quantity
            else:
                new_orders_in_transit.append((arrival_day, order_quantity))
        orders_in_transit = new_orders_in_transit

        # 3. Subtract the sampled daily demand
        current_inventory -= sampled_demand

        is_stockout = False
        if current_inventory < 0:
            is_stockout = True
            current_inventory = 0 # Inventory cannot go below zero physically

        # 4. Check for reorder
        if current_inventory <= reorder_point:
            # Check if an order for EOQ is already in transit.
            # Assume only one EOQ order can be pending at a time to simplify.
            order_pending = False
            for arrival_day, _ in orders_in_transit:
                if arrival_day > day: # An order placed previously is still in transit
                    order_pending = True
                    break
            if not order_pending:
                # Place a new order, it will arrive after 'lead_time' days
                orders_in_transit.append((day + lead_time, eoq))

        # 5. Record the day's events
        time_points.append(day)
        inventory_levels.append(current_inventory)
        stockout_flags.append(is_stockout)

    return time_points, inventory_levels, stockout_flags

In [61]:
# @title [Visualize]

import math
import altair as alt
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display

# Define output_widget here to ensure it's available
output_widget = widgets.Output()

# Initialize global variables to store the trajectory data for later use
global_time_points = []
global_inventory_levels = []
global_stockout_flags = [] # New global variable for stockout flags
global_current_eoq = 0.0 # To store the EOQ value for other steps

def update_eoq_metrics_and_trajectory(demand, holding_cost, ordering_cost, sd_demand, lead_time):
    global global_time_points, global_inventory_levels, global_current_eoq, global_stockout_flags

    with output_widget:
        output_widget.clear_output() # Clear previous output, including plots

        if demand <= 0 or holding_cost <= 0 or ordering_cost <= 0 or sd_demand < 0 or lead_time < 0:
            print("All input values must be positive, except for standard deviation which can be zero.")
            global_current_eoq = 0.0
            global_time_points = []
            global_inventory_levels = []
            global_stockout_flags = []
            return

        # --- Uncertainty Calculations ---
        daily_demand = demand / 365 # Assuming 365 days in a year
        sd_daily_demand = sd_demand / math.sqrt(365) # Assuming 365 days in a year

        # Calculate Std Dev Demand During Lead Time
        std_dev_demand_lead_time = sd_daily_demand * math.sqrt(lead_time) if lead_time >= 0 else 0.0

        z_score = 1.645 # For 95% service level
        safety_stock = z_score * std_dev_demand_lead_time
        reorder_point = (daily_demand * lead_time) + safety_stock

        # Calculate EOQ
        eoq = math.sqrt((2 * demand * ordering_cost) / holding_cost)
        global_current_eoq = eoq # Store EOQ globally for potential future use

        # Calculate Number of orders per year
        num_orders_per_year = demand / eoq

        # Calculate Total annual holding cost
        total_annual_holding_cost = (eoq / 2) * holding_cost

        # Calculate Total annual ordering cost
        total_annual_ordering_cost = (demand / eoq) * ordering_cost

        # Calculate Total annual inventory cost
        total_annual_inventory_cost = total_annual_holding_cost + total_annual_ordering_cost

        # Display the calculated metrics
        print(f"Economic Order Quantity (EOQ): {eoq:.2f}")
        print(f"Number of Orders per Year: {num_orders_per_year:.2f}")
        print(f"Total Annual Holding Cost: {total_annual_holding_cost:.2f}")
        print(f"Total Annual Ordering Cost: {total_annual_ordering_cost:.2f}")
        print(f"Total Annual Inventory Cost: {total_annual_inventory_cost:.2f}")

        # Display Uncertainty Metrics
        print(f"\n--- Uncertainty Metrics (95% Service Level) ---")
        print(f"Average Daily Demand: {daily_demand:.2f}")
        print(f"Std Dev Daily Demand: {sd_daily_demand:.2f}")
        print(f"Std Dev Demand During Lead Time: {std_dev_demand_lead_time:.2f}")
        print(f"Z-score (95% Service Level): {z_score:.3f}")
        print(f"Safety Stock: {safety_stock:.2f}")
        print(f"Reorder Point: {reorder_point:.2f}")

        # Generate inventory trajectory using the calculated EOQ and uncertainty metrics
        time_points, inventory_levels, stockout_flags = generate_inventory_trajectory(eoq, daily_demand, sd_daily_demand, lead_time, reorder_point)

        # Store the trajectory data globally so it can be accessed for plotting
        global_time_points = time_points
        global_inventory_levels = inventory_levels
        global_stockout_flags = stockout_flags # Store stockout flags

        print(f"\nInventory trajectory data generated with {len(time_points)} data points.")

        # --- Generate and display the cost curves plot ---
        print("\n--- Inventory Cost Curves Plot ---")
        # Generate a range of Q values around EOQ
        q_min = max(1, int(eoq * 0.1))
        q_max = int(eoq * 3.0)
        q_values = np.linspace(q_min, q_max, 500)

        cost_data = []
        for q in q_values:
            if q > 0: # Ensure Q is positive to avoid division by zero
                holding_c = (q / 2) * holding_cost
                ordering_c = (demand / q) * ordering_cost
                total_c = holding_c + ordering_c
                cost_data.append({'Order Quantity (Q)': q, 'Cost Type': 'Annual Holding Cost', 'Cost': holding_c})
                cost_data.append({'Order Quantity (Q)': q, 'Cost Type': 'Annual Ordering Cost', 'Cost': ordering_c})
                cost_data.append({'Order Quantity (Q)': q, 'Cost Type': 'Total Annual Inventory Cost', 'Cost': total_c})

        cost_df = pd.DataFrame(cost_data)

        base_costs = alt.Chart(cost_df).encode(
            x=alt.X('Order Quantity (Q):Q', title='Order Quantity (Q)'),
            y=alt.Y('Cost:Q', title='Cost'),
            color=alt.Color('Cost Type:N', title='Cost Type')
        ).properties(
            title='Inventory Cost Curves vs. Order Quantity',
            width='container'
        )

        # Line chart for costs
        line_costs = base_costs.mark_line().encode(
            tooltip=[
                alt.Tooltip('Order Quantity (Q)', format='.1f'),
                alt.Tooltip('Cost Type'),
                alt.Tooltip('Cost', format='.2f')
            ]
        )

        # Add a point for EOQ on the total cost curve
        eoq_point = alt.Chart(pd.DataFrame({
            'Order Quantity (Q)': [eoq],
            'Cost Type': ['Total Annual Inventory Cost'],
            'Cost': [total_annual_inventory_cost]
        })).mark_point(color='red', size=100, filled=True, shape='diamond').encode(
            x=alt.X('Order Quantity (Q):Q'),
            y=alt.Y('Cost:Q'),
            tooltip=[
                alt.Tooltip('Order Quantity (Q)', format='.1f', title='EOQ'),
                alt.Tooltip('Cost', format='.2f', title='Min Total Cost')
            ]
        )

        # Add a vertical rule at EOQ
        eoq_rule = alt.Chart(pd.DataFrame({'EOQ': [eoq]})).mark_rule(color='red', strokeDash=[3,3]).encode(
            x='EOQ:Q',
            tooltip=[alt.Tooltip('EOQ', format='.1f', title='EOQ')]
        )

        chart_costs = (line_costs + eoq_rule + eoq_point).interactive()
        display(chart_costs)

        # --- Re-generate and display the animated plot ---
        print("\n--- Animated Inventory Trajectory Plot ---")
        inventory_df = pd.DataFrame({
            'Time': global_time_points,
            'Inventory Level': global_inventory_levels,
            'Is Stockout': global_stockout_flags # Include stockout flags
        })

        base_animated = alt.Chart(inventory_df).encode(
            x=alt.X('Time', title='Time (Days)', scale=alt.Scale(domain=[0, inventory_df['Time'].max() if not inventory_df.empty else 0])),
            y=alt.Y('Inventory Level', title='Inventory Level', scale=alt.Scale(domain=[min(0, inventory_df['Inventory Level'].min() if not inventory_df.empty else 0), inventory_df['Inventory Level'].max() if not inventory_df.empty else 0]))
        ).properties(
            title='Simulated Inventory Trajectory Over Time',
            width='container'
        )

        line_animated = base_animated.mark_line(color='blue').encode()

        # Reorder Point Line
        reorder_point_line = alt.Chart(pd.DataFrame({'value': [reorder_point]})).mark_rule(color='orange', strokeDash=[5,5], size=1).encode(
            y=alt.Y('value:Q', title='Inventory Level'),
            tooltip=[alt.Tooltip('value', format='.2f', title='Reorder Point')]
        )

        # Safety Stock Line
        safety_stock_line = alt.Chart(pd.DataFrame({'value': [safety_stock]})).mark_rule(color='purple', strokeDash=[5,5], size=1).encode(
            y=alt.Y('value:Q', title='Inventory Level'),
            tooltip=[alt.Tooltip('value', format='.2f', title='Safety Stock')]
        )

        # Stockout Highlight
        stockout_highlight = alt.Chart(inventory_df[inventory_df['Is Stockout'] == True]).mark_point(color='red', size=100, shape='cross').encode(
            x='Time:Q',
            y='Inventory Level:Q',
            tooltip=[
                alt.Tooltip('Time', format='.1f'),
                alt.Tooltip('Inventory Level', format='.1f'),
                alt.Tooltip('Is Stockout', title='Stockout')
            ]
        )

        slider_value = alt.param(
            name='time_param',
            value=0,
            bind=alt.binding_range(min=0, max=inventory_df['Time'].max() if not inventory_df.empty else 0, step=1, name='Time:')
        )

        animation_dot = base_animated.mark_circle(size=100, color='red').encode(
            opacity=alt.value(1),
            tooltip=[
                alt.Tooltip('Time', format='.1f'),
                alt.Tooltip('Inventory Level', format='.1f'),
                alt.Tooltip('Is Stockout', title='Stockout') # Updated tooltip
            ]
        ).transform_calculate(
            distance=f"abs(datum.Time - time_param)"
        ).transform_window(
            rank="rank()",
            sort=[alt.EncodingSortField("distance", order="ascending")]
        ).transform_filter(
            alt.datum.rank == 1
        )

        animated_chart = (line_animated + reorder_point_line + safety_stock_line + stockout_highlight + animation_dot).add_params(
            slider_value
        ).interactive().properties(
            width='container'
        )
        display(animated_chart)


# Re-bind the interactive output to the updated function that includes trajectory generation
# This replaces the previous `interactive_eoq_output` binding
interactive_output_full = widgets.interactive_output(
    update_eoq_metrics_and_trajectory,
    {
        'demand': demand_widget,
        'holding_cost': holding_cost_widget,
        'ordering_cost': ordering_cost_widget,
        'sd_demand': sd_widget,
        'lead_time': leadtime_widget
    }
)

# Re-displaying `output_widget` and the new `interactive_output_full` ensures the new binding is active and visible.
display(output_widget, interactive_output_full)
print("Interactive EOQ metrics display with uncertainty calculations and trajectory plot updated successfully.")

Output()

Output()

Interactive EOQ metrics display with uncertainty calculations and trajectory plot updated successfully.
